In [1]:
from functools import reduce
from json import loads
from time import sleep

import pandas as pd
import requests
from fake_useragent import UserAgent
from tqdm import tqdm

In [2]:
ua = UserAgent()
urls = [
    "https://web.archive.org/web/20240618223813/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-09-2024-15",
    "https://web.archive.org/web/20240618224001/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-01-2024-07",
    "https://web.archive.org/web/20240627224113/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-31-2024-07",
    "https://web.archive.org/web/20240607014131/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-24-2024-17",
    "https://web.archive.org/web/20240520101213/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-20-2024-10",
    "https://web.archive.org/web/20240513083145/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-13-2024-08",
    "https://web.archive.org/web/20240512055402/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-12-2024-05",
    "https://web.archive.org/web/20240820084557/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-09-2024-23",
    "https://web.archive.org/web/20240808194954/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-08-2024-08",
    "https://web.archive.org/web/20240829062530/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-03-2024-17",
    "https://web.archive.org/web/20240507133357/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-01-2024-02",
    "https://web.archive.org/web/20240425024738/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-25-2024-02",
    "https://web.archive.org/web/20240421200031/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-21-2024-20",
    "https://web.archive.org/web/20240607014508/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-16-2024-01",
    "https://web.archive.org/web/20240724082736/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-12-2024-14",
    "https://web.archive.org/web/20240413074519/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-05-2024-13",
    "https://web.archive.org/web/20240402105454/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-02-2024-10",
    "https://web.archive.org/web/20240310072026/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-10-2024-07",
    "https://web.archive.org/web/20240401140330/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-08-2024-08",
    "https://web.archive.org/web/20240304192623/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-04-2024-19",
    "https://web.archive.org/web/20240221210414/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-21-2024-21",
    "https://web.archive.org/web/20240213004635/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-13-2024-00",
    "https://web.archive.org/web/20240206022033/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-06-2024-02",
    "https://web.archive.org/web/20240122140735/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-22-2024-14",
    "https://web.archive.org/web/20240120141454/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-20-2024-14",
    "https://web.archive.org/web/20240116090127/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-08-2024-18",
    "https://web.archive.org/web/20240104004433/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-04-2024-00",
    "https://web.archive.org/web/20240114193327/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-02-2024-09",
    "https://web.archive.org/web/20231226121459/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-26-2023-12",
    "https://web.archive.org/web/20231223083743/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-23-2023-08",
    "https://web.archive.org/web/20231220224849/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-20-2023-22",
    "https://web.archive.org/web/20231127192403/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-27-2023-19",
    "https://web.archive.org/web/20231229092809/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-16-2023-19",
    "https://web.archive.org/web/20231114235145/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-14-2023-23",
    "https://web.archive.org/web/20231024164809/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-24-2023-16",
    "https://web.archive.org/web/20231011024544/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-11-2023-02",
    "https://web.archive.org/web/20231009181328/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-07-2023-07",
    "https://web.archive.org/web/20231002144841/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-02-2023-14",
    "https://web.archive.org/web/20231229093349/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-30-2023-03",
    "https://web.archive.org/web/20230929022618/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-29-2023-02",
    "https://web.archive.org/web/20240818112411/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-27-2023-06",
    "https://web.archive.org/web/20231229093546/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-19-2023-23",
    "https://web.archive.org/web/20230916021519/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-16-2023-02",
    "https://web.archive.org/web/20230904124533/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-02-2023-16",
    "https://web.archive.org/web/20230820221229/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-20-2023-22",
    "https://web.archive.org/web/20230813212256/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-13-2023-21",
    "https://web.archive.org/web/20231223040320/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-06-2023-11",
    "https://web.archive.org/web/20240823192815/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-04-2023-17",
    "https://web.archive.org/web/20230727092600/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-23-2023-20",
    "https://web.archive.org/web/20230712231029/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-12-2023-23",
    "https://web.archive.org/web/20230701154155/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-01-2023-15",
    "https://web.archive.org/web/20231117134008/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-01-2023-05",
    "https://web.archive.org/web/20230629020916/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-16-2023-04",
    "https://web.archive.org/web/20230613221113/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-13-2023-22",
    "https://web.archive.org/web/20230609185355/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-09-2023-18",
    "https://web.archive.org/web/20231117135212/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-28-2023-09",
    "https://web.archive.org/web/20230526031308/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-26-2023-03",
    "https://web.archive.org/web/20231117224520/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-21-2023-19",
    "https://web.archive.org/web/20231022083338/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-12-2023-23",
    "https://web.archive.org/web/20230712163534/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-06-2023-11",
    "https://web.archive.org/web/20231117135522/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-07-2023-14",
    "https://web.archive.org/web/20230406081929/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-06-2023-08",
    "https://web.archive.org/web/20240306152510/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-04-2023-21",
    "https://web.archive.org/web/20230712163841/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-29-2023-21",
    "https://web.archive.org/web/20230325052233/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-25-2023-05",
    "https://web.archive.org/web/20231115134331/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-15-2023-16",
    "https://web.archive.org/web/20240415102702/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-12-2023-14",
    "https://web.archive.org/web/20230302000420/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-02-2023-00",
    "https://web.archive.org/web/20230216035946/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-16-2023-03",
    "https://web.archive.org/web/20230210153353/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-10-2023-15",
    "https://web.archive.org/web/20230626233724/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-27-2023-11",
    "https://web.archive.org/web/20230121221150/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-21-2023-22",
    "https://web.archive.org/web/20230123005207/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-07-2023-15",
    "https://web.archive.org/web/20221214124948/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-14-2022-12",
    "https://web.archive.org/web/20221206170529/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-06-2022-17",
    "https://web.archive.org/web/20221212164015/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-22-2022-12",
    "https://web.archive.org/web/20221111052627/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-11-2022-05",
    "https://web.archive.org/web/20221104142840/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-04-2022-14",
    "https://web.archive.org/web/20221019125237/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-19-2022-12",
    "https://web.archive.org/web/20221212164827/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-18-2022-01",
    "https://web.archive.org/web/20221015134237/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-15-2022-00",
    "https://web.archive.org/web/20221006114406/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-06-2022-11",
    "https://web.archive.org/web/20220923021059/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-23-2022-02",
    "https://web.archive.org/web/20220922224633/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-17-2022-18",
    "https://web.archive.org/web/20220911141747/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-11-2022-14",
    "https://web.archive.org/web/20220904021736/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-04-2022-02",
    "https://web.archive.org/web/20220905122733/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-22-2022-08",
    "https://web.archive.org/web/20220702044420/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-02-2022-04",
    "https://web.archive.org/web/20220613042030/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-13-2022-04",
    "https://web.archive.org/web/20220806180820/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-28-2022-07",
    "https://web.archive.org/web/20221216084338/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-26-2022-15",
    "https://web.archive.org/web/20220525114317/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-25-2022-11",
    "https://web.archive.org/web/20220801211811/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-21-2022-19",
    "https://web.archive.org/web/20220519230426/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-14-2022-04",
    "https://web.archive.org/web/20220512202119/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-12-2022-20",
    "https://web.archive.org/web/20220510124032/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-10-2022-12",
    "https://web.archive.org/web/20220507054343/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-07-2022-05",
    "https://web.archive.org/web/20220506125145/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-02-2022-13",
    "https://web.archive.org/web/20220426055219/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-26-2022-05",
    "https://web.archive.org/web/20220805212738/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-24-2022-22",
    "https://web.archive.org/web/20220805212803/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-19-2022-17",
    "https://web.archive.org/web/20220409235005/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-04-2022-03",
    "https://web.archive.org/web/20220409233739/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-02-2022-09",
    "https://web.archive.org/web/20220409233914/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-17-2022-19",
    "https://web.archive.org/web/20220308091519/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-08-2022-09",
    "https://web.archive.org/web/20220409234607/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-07-2022-09",
    "https://web.archive.org/web/20220303051123/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-03-2022-05",
    "https://web.archive.org/web/20220301151904/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-01-2022-15",
    "https://web.archive.org/web/20220322081023/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-28-2022-12",
    "https://web.archive.org/web/20220226021303/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-26-2022-02",
    "https://web.archive.org/web/20220805213312/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-23-2022-00",
    "https://web.archive.org/web/20220219141126/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-19-2022-14",
    "https://web.archive.org/web/20220214105200/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-14-2022-10",
    "https://web.archive.org/web/20220211161606/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-11-2022-16",
    "https://web.archive.org/web/20220208170653/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-08-2022-17",
    "https://web.archive.org/web/20220204062607/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-04-2022-06",
    "https://web.archive.org/web/20220803011525/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-03-2022-06",
    "https://web.archive.org/web/20220130133804/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-30-2022-13",
    "https://web.archive.org/web/20250105074839/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-23-2022-08",
    "https://web.archive.org/web/20220322151100/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-19-2022-21",
    "https://web.archive.org/web/20220117144531/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-17-2022-14",
    "https://web.archive.org/web/20220115063522/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-15-2022-06",
    "https://web.archive.org/web/20220114015646/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-14-2022-01",
    "https://web.archive.org/web/20220112091632/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-12-2022-09",
    "https://web.archive.org/web/20220107042907/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-07-2022-04",
    "https://web.archive.org/web/20220102141912/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-02-2022-14",
    "https://web.archive.org/web/20211228113054/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-28-2021-11",
    "https://web.archive.org/web/20211227071810/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-27-2021-07",
    "https://web.archive.org/web/20220101141248/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-21-2021-09",
    "https://web.archive.org/web/20220114005648/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-18-2021-00",
    "https://web.archive.org/web/20211226121510/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-15-2021-10",
    "https://web.archive.org/web/20211208191129/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-08-2021-19",
    "https://web.archive.org/web/20221216080933/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-01-2021-07",
    "https://web.archive.org/web/20211125045705/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-17-2021-18",
    "https://web.archive.org/web/20211116071623/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-15-2021-09",
    "https://web.archive.org/web/20211116110604/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-04-2021-20",
    "https://web.archive.org/web/20211205084340/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-03-2021-07",
    "https://web.archive.org/web/20211024114822/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-13-2021-10",
    "https://web.archive.org/web/20210928160208/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-28-2021-16",
    "https://web.archive.org/web/20211125050013/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-24-2021-18",
    "https://web.archive.org/web/20210922145301/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-22-2021-14",
    "https://web.archive.org/web/20220410154254/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-17-2021-02",
    "https://web.archive.org/web/20220805214714/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-13-2021-02",
    "https://web.archive.org/web/20210905184059/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-05-2021-18",
    "https://web.archive.org/web/20210904195639/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-24-2021-01",
    "https://web.archive.org/web/20210904043810/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-14-2021-22",
    "https://web.archive.org/web/20210904043523/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-13-2021-14",
    "https://web.archive.org/web/20210904043231/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-02-2021-23",
    "https://web.archive.org/web/20210720175953/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-20-2021-17",
    "https://web.archive.org/web/20210904044317/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-15-2021-23",
    "https://web.archive.org/web/20220417133707/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-12-2021-20",
    "https://web.archive.org/web/20220102001735/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-08-2021-07",
    "https://web.archive.org/web/20210408112249/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-02-2021-17",
    "https://web.archive.org/web/20210307042215/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-07-2021-04",
    "https://web.archive.org/web/20210307042221/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-19-2021-01",
    "https://web.archive.org/web/20210307042238/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-15-2021-22",
    "https://web.archive.org/web/20210212124317/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-12-2021-12",
    "https://web.archive.org/web/20210121145727/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-15-2021-11",
    "https://web.archive.org/web/20201225184539/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-25-2020-18",
    "https://web.archive.org/web/20201224182122/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-24-2020-18",
    "https://web.archive.org/web/20201223181559/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-23-2020-18",
    "https://web.archive.org/web/20201219172623/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-19-2020-17",
    "https://web.archive.org/web/20201215075138/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-15-2020-07",
    "https://web.archive.org/web/20201213124353/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-13-2020-12",
    "https://web.archive.org/web/20201211000852/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-11-2020-00",
    "https://web.archive.org/web/20201209232909/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-09-2020-23",
    "https://web.archive.org/web/20201208225018/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-08-2020-22",
    "https://web.archive.org/web/20201207175338/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-07-2020-17",
    "https://web.archive.org/web/20201129065514/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-29-2020-06",
    "https://web.archive.org/web/20211125072559/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-26-2020-10",
    "https://web.archive.org/web/20201121151815/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-18-2020-06",
    "https://web.archive.org/web/20201113055041/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-13-2020-05",
    "https://web.archive.org/web/20201111011524/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-11-2020-01",
    "https://web.archive.org/web/20201210110552/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-08-2020-13",
    "https://web.archive.org/web/20220805220113/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-04-2020-21",
    "https://web.archive.org/web/20211116072736/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-02-2020-19",
    "https://web.archive.org/web/20201024165916/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-24-2020-16",
    "https://web.archive.org/web/20201021053929/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-21-2020-05",
    "https://web.archive.org/web/20201018050324/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-08-2020-09",
    "https://web.archive.org/web/20201018050345/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-26-2020-23",
    "https://web.archive.org/web/20200923175409/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-09-2020-09",
    "https://web.archive.org/web/20200923175359/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-06-2020-11",
    "https://web.archive.org/web/20200923175139/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-26-2020-00",
    "https://web.archive.org/web/20200923175056/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-19-2020-15",
    "https://web.archive.org/web/20200817214836/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-22-2020-23",
    "https://web.archive.org/web/20200911031821/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-02-2020-22",
    "https://web.archive.org/web/20201208034552/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-02-2020-09",
    "https://web.archive.org/web/20200615161608/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-09-2020-09",
    "https://web.archive.org/web/20200617132746/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-07-2020-21",
    "https://web.archive.org/web/20200526142659/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-26-2020-14",
    "https://web.archive.org/web/20200630164452/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-18-2020-07",
    "https://web.archive.org/web/20200517070912/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-17-2020-07",
    "https://web.archive.org/web/20200608153330/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-14-2020-14",
    "https://web.archive.org/web/20200513052633/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-13-2020-05",
    "https://web.archive.org/web/20200425030440/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-25-2020-03",
    "https://web.archive.org/web/20200428164239/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-23-2020-12",
    "https://web.archive.org/web/20200429153436/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-19-2020-16",
    "https://web.archive.org/web/20200403220311/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-03-2020-22",
    "https://web.archive.org/web/20200409171644/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-02-2020-14",
    "https://web.archive.org/web/20200328003343/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-25-2020-06",
    "https://web.archive.org/web/20200320064831/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-20-2020-06",
    "https://web.archive.org/web/20200320142435/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-18-2020-15",
    "https://web.archive.org/web/20200317210128/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-06-2020-03",
    "https://web.archive.org/web/20200317210139/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-05-2020-12",
    "https://web.archive.org/web/20191227095912/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-27-2019-09",
    "https://web.archive.org/web/20191213002732/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-13-2019-00",
    "https://web.archive.org/web/20191204061815/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-04-2019-06",
    "https://web.archive.org/web/20191201034554/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-01-2019-03",
    "https://web.archive.org/web/20191121121530/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-12-2019-01",
    "https://web.archive.org/web/20191018095226/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-14-2019-17",
    "https://web.archive.org/web/20191018095716/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-10-2019-08",
    "https://web.archive.org/web/20191002050206/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-02-2019-05",
    "https://web.archive.org/web/20190926061638/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-10-2019-02",
    "https://web.archive.org/web/20190728002440/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-17-2019-01",
    "https://web.archive.org/web/20190624134107/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-24-2019-13",
    "https://web.archive.org/web/20190715100955/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-16-2019-06",
    "https://web.archive.org/web/20190524002602/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-24-2019-00",
    "https://web.archive.org/web/20190524002617/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-21-2019-17",
    "https://web.archive.org/web/20190524002641/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-08-2019-00",
    "https://web.archive.org/web/20220805222617/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-26-2019-14",
    "https://web.archive.org/web/20190423135346/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-17-2019-01",
    "https://web.archive.org/web/20190602154725/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-14-2019-01",
    "https://web.archive.org/web/20220806173748/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-06-2019-17",
    "https://web.archive.org/web/20190503182305/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-02-2019-19",
    "https://web.archive.org/web/20191106112548/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-21-2019-08",
    "https://web.archive.org/web/20190503182942/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-02-2019-12",
    "https://web.archive.org/web/20190328214617/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-14-2019-15",
    "https://web.archive.org/web/20191121124435/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-05-2019-18",
    "https://web.archive.org/web/20191105140951/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-02-2019-01",
    "https://web.archive.org/web/20190204205939/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-30-2019-19",
    "https://web.archive.org/web/20191121124433/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-28-2019-13",
    "https://web.archive.org/web/20190204210013/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-28-2019-06",
    "https://web.archive.org/web/20190204210052/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-22-2019-05",
    "https://web.archive.org/web/20190204210212/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-19-2019-19",
    "https://web.archive.org/web/20190204210237/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-16-2019-00",
    "https://web.archive.org/web/20190503185950/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-05-2019-22",
    "https://web.archive.org/web/20190124230816/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-30-2018-07",
    "https://web.archive.org/web/20190124230927/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-27-2018-14",
    "https://web.archive.org/web/20210107154333/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-23-2018-00",
    "https://web.archive.org/web/20190124230742/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-22-2018-06",
    "https://web.archive.org/web/20181204000937/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-04-2018-00",
    "https://web.archive.org/web/20191210172252/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-28-2018-10",
    "https://web.archive.org/web/20191126101508/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-22-2018-07",
    "https://web.archive.org/web/20181117194324/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-17-2018-19",
    "https://web.archive.org/web/20181117194118/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-04-2018-08",
    "https://web.archive.org/web/20190328214728/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-21-2018-23",
    "https://web.archive.org/web/20190315172514/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-21-2018-00",
    "https://web.archive.org/web/20180909234102/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-02-2018-07",
    "https://web.archive.org/web/20180909233054/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-23-2018-14",
    "https://web.archive.org/web/20180907062032/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-16-2018-13",
    "https://web.archive.org/web/20180811111522/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-11-2018-11",
    "https://web.archive.org/web/20180809003810/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-02-2018-04",
    "https://web.archive.org/web/20180730220728/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-26-2018-23",
    "https://web.archive.org/web/20180723233053/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-11-2018-06",
    "https://web.archive.org/web/20180713202104/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-24-2018-14",
    "https://web.archive.org/web/20180713202210/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-13-2018-21",
    "https://web.archive.org/web/20180705150514/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-26-2018-10",
    "https://web.archive.org/web/20220802195517/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-20-2018-16",
    "https://web.archive.org/web/20180508235655/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-08-2018-23",
    "https://web.archive.org/web/20180511014927/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-26-2018-02",
    "https://web.archive.org/web/20180511014928/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-23-2018-08",
    "https://web.archive.org/web/20180511014925/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-18-2018-03",
    "https://web.archive.org/web/20180427115434/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-16-2018-19",
    "https://web.archive.org/web/20180511021012/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-09-2018-09",
    "https://web.archive.org/web/20181031185028/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-02-2018-22",
    "https://web.archive.org/web/20180518181048/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-01-2018-07",
    "https://web.archive.org/web/20180328151838/https://steamcdn-a.akamaihd.net/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-28-2018-15",
    "https://web.archive.org/web/20180419060852/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-21-2018-06",
    "https://web.archive.org/web/20180311235615/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-09-2018-08",
    "https://web.archive.org/web/20180306012653/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-02-2018-20",
    "https://web.archive.org/web/20190322145048/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-28-2018-09",
    "https://web.archive.org/web/20180218113536/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-16-2018-09",
    "https://web.archive.org/web/20180212232259/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-10-2018-08",
    "https://web.archive.org/web/20220806181611/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-04-2018-01",
    "https://web.archive.org/web/20220806175250/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-02-2018-18",
    "https://web.archive.org/web/20180131180157/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-31-2018-18",
    "https://web.archive.org/web/20220806175729/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-28-2018-15",
    "https://web.archive.org/web/20180216023615/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-24-2018-06",
    "https://web.archive.org/web/20180221201109/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-10-2018-21",
    "https://web.archive.org/web/20180107070259/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-03-2018-01",
    "https://web.archive.org/web/20180218044806/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-01-2018-22",
    "https://web.archive.org/web/20171223180818/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-22-2017-07",
    "https://web.archive.org/web/20190602161333/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-19-2017-02",
    "https://web.archive.org/web/20180216023426/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-15-2017-22",
    "https://web.archive.org/web/20171216041131/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-14-2017-03",
    "https://web.archive.org/web/20171220010943/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-09-2017-20",
    "https://web.archive.org/web/20180221201309/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-08-2017-11",
    "https://web.archive.org/web/20171207105741/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-05-2017-19",
    "https://web.archive.org/web/20171218235817/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-02-2017-20",
    "https://web.archive.org/web/20180221201333/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-29-2017-03",
    "https://web.archive.org/web/20171128002855/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-26-2017-10",
    "https://web.archive.org/web/20171123031423/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-20-2017-21",
    "https://web.archive.org/web/20210215181850/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-16-2017-21",
    "https://web.archive.org/web/20171120155601/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-12-2017-14",
    "https://web.archive.org/web/20171111155422/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-10-2017-08",
    "https://web.archive.org/web/20180211182103/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-06-2017-10",
    "https://web.archive.org/web/20171101044541/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-30-2017-01",
    "https://web.archive.org/web/20180511014133/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-25-2017-13",
    "https://web.archive.org/web/20191029104929/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-22-2017-15",
    "https://web.archive.org/web/20191029105108/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-15-2017-02",
    "https://web.archive.org/web/20191029105123/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-13-2017-07",
    "https://web.archive.org/web/20171026012653/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-08-2017-04",
    "https://web.archive.org/web/20170928162605/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-27-2017-17",
    "https://web.archive.org/web/20180511013743/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-25-2017-17",
    "https://web.archive.org/web/20180511013741/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-22-2017-11",
    "https://web.archive.org/web/20170917111041/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-16-2017-13",
    "https://web.archive.org/web/20210103094648/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-13-2017-19",
    "https://web.archive.org/web/20170910200348/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-09-2017-11",
    "https://web.archive.org/web/20170908220935/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-07-2017-23",
    "https://web.archive.org/web/20170901191306/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-31-2017-19",
    "https://web.archive.org/web/20240520165248/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-30-2017-16",
    "https://web.archive.org/web/20170831105504/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-26-2017-01",
    "https://web.archive.org/web/20180222193812/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-22-2017-02",
    "https://web.archive.org/web/20170817134518/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-15-2017-20",
    "https://web.archive.org/web/20180221201502/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-13-2017-02",
    "https://web.archive.org/web/20170921171854/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-08-2017-15",
    "https://web.archive.org/web/20170803212854/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-02-2017-07",
    "https://web.archive.org/web/20170728213404/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-27-2017-06",
    "https://web.archive.org/web/20180718205348/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-16-2017-17",
    "https://web.archive.org/web/20180221232406/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-10-2017-23",
    "https://web.archive.org/web/20180221232438/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-28-2017-18",
    "https://web.archive.org/web/20180511015916/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-27-2017-11",
    "https://web.archive.org/web/20180114104518/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-24-2017-05",
    "https://web.archive.org/web/20180216022450/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-20-2017-10",
    "https://web.archive.org/web/20180221233330/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-18-2017-13",
    "https://web.archive.org/web/20180222194447/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-16-2017-12",
    "https://web.archive.org/web/20170613201633/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-07-2017-17",
    "https://web.archive.org/web/20180222210009/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-31-2017-20",
    "https://web.archive.org/web/20170531070129/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-29-2017-14",
    "https://web.archive.org/web/20180221233647/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-27-2017-21",
    "https://web.archive.org/web/20170522223808/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-21-2017-02",
    "https://web.archive.org/web/20170521100241/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=05-18-2017-22",
    "https://web.archive.org/web/20171113230832/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=04-22-2017-14",
    "https://web.archive.org/web/20171013172043/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-20-2017-08",
    "https://web.archive.org/web/20170918074351/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-18-2017-03",
    "https://web.archive.org/web/20180130165900/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=03-02-2017-07",
    "https://web.archive.org/web/20170921171853/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=02-15-2017-15",
    "https://web.archive.org/web/20180215015353/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-20-2017-05",
    "https://web.archive.org/web/20170921171744/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-18-2017-20",
    "https://web.archive.org/web/20180511015904/http://cdn.edgecast.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=01-15-2017-00",
    "https://web.archive.org/web/20170929012050/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-25-2016-01",
    "https://web.archive.org/web/20161213004239/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-13-2016-00",
    "https://web.archive.org/web/20180427115352/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-02-2016-05",
    "https://web.archive.org/web/20180221234025/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-23-2016-06",
    "https://web.archive.org/web/20221101064429/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-19-2016-05",
    "https://web.archive.org/web/20161208173425/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-10-2016-09",
    "https://web.archive.org/web/20180304163706/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=11-01-2016-19",
    "https://web.archive.org/web/20180221234153/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-29-2016-01",
    "https://web.archive.org/web/20170921205432/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-14-2016-20",
    "https://web.archive.org/web/20161010031044/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-08-2016-05",
    "https://web.archive.org/web/20180212175823/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-05-2016-07",
    "https://web.archive.org/web/20161005044106/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=10-01-2016-17",
    "https://web.archive.org/web/20200410164149/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-30-2016-13",
    "https://web.archive.org/web/20180216021815/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-27-2016-11",
    "https://web.archive.org/web/20190516061049/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-07-2016-06",
    "https://web.archive.org/web/20171013172907/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=09-02-2016-10",
    "https://web.archive.org/web/20191117034531/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-31-2016-05",
    "https://web.archive.org/web/20171013173113/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-27-2016-19",
    "https://web.archive.org/web/20160826102911/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-24-2016-19",
    "https://web.archive.org/web/20160820225246/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-15-2016-14",
    "https://web.archive.org/web/20160811205336/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=08-09-2016-20",
    "https://web.archive.org/web/20180221234607/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-30-2016-09",
    "https://web.archive.org/web/20180221234634/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-26-2016-04",
    "https://web.archive.org/web/20160726200431/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-24-2016-15",
    "https://web.archive.org/web/20160725081055/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-23-2016-07",
    "https://web.archive.org/web/20160722202926/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-19-2016-23",
    "https://web.archive.org/web/20160716194340/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-14-2016-00",
    "https://web.archive.org/web/20160711064115/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-08-2016-17",
    "https://web.archive.org/web/20160705040758/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=07-02-2016-22",
    "https://web.archive.org/web/20160719111434/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-26-2016-23",
    "https://web.archive.org/web/20180221234835/http://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-25-2016-04",
]

In [4]:
all_dfs = []
for url in tqdm(urls):
    headers = {"User-Agent": ua.random}
    response = requests.get(url, headers=headers, timeout=None)
    startidx = response.text.find("(")
    endidx = response.text.find(")")
    data = loads(response.text[startidx + 1 : endidx])

    if "json" not in data:
        continue

    series_list = loads(data["json"])

    df_list = []
    for series in series_list:
        df_dict = {}
        region = series["label"]
        df_dict["Timestamp"] = [
            pd.to_datetime(x[0], unit="ms") for x in series["data"]
        ]
        df_dict[region] = [int(x[1]) for x in series["data"]]
        df_list.append(pd.DataFrame(df_dict))

    df = reduce(
        lambda x, y: pd.merge(x, y, on="Timestamp", how="outer"), df_list
    )
    df = df.sort_values("Timestamp").reset_index(drop=True)

    all_dfs.append(df)

    sleep(60)

df_combined = pd.concat(all_dfs, axis=0, ignore_index=True)
df_combined.to_csv("../data/old_data.csv", index=False)

100%|██████████| 371/371 [6:19:14<00:00, 61.33s/it]  


,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2024-06-07 15:00:00,47.0,121.0,764.0,293.0,986.0,1480.0,11580,6451,2957.0
1,2024-06-07 15:10:00,47.0,118.0,762.0,285.0,986.0,1480.0,11316,6451,2957.0
2,2024-06-07 15:20:00,50.0,120.0,764.0,285.0,1002.0,1506.0,11316,6490,2969.0
3,2024-06-07 15:30:00,52.0,120.0,779.0,274.0,1040.0,1520.0,11053,6603,3029.0
4,2024-06-07 15:40:00,52.0,122.0,779.0,265.0,1051.0,1527.0,10653,6694,3108.0
...,...,...,...,...,...,...,...,...,...,...
168432,2018-02-21 21:05:20,11.0,10.0,83.0,50.0,361.0,239.0,380,1529,1167.0
168433,2018-02-21 21:10:20,11.0,10.0,83.0,50.0,360.0,234.0,378,1502,1179.0
168434,2018-02-21 21:15:20,11.0,10.0,81.0,50.0,360.0,231.0,377,1468,1192.0
168435,2018-02-21 21:20:20,11.0,10.0,79.0,50.0,361.0,228.0,372,1433,1196.0


In [5]:
df_combined = df_combined.sort_values("Timestamp").reset_index(drop=True)
df_combined.to_csv("../data/old_data.csv", index=False)

In [7]:
df_combined_10min = df_combined.loc[(df_combined["Timestamp"].dt.minute % 10 == 0) & (df_combined["Timestamp"].dt.second == 0)]
df_combined_10min

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
5825,2016-10-03 10:20:00,2.0,5.0,29.0,55.0,31.0,153.0,386,465,142.0
5827,2016-10-03 10:30:00,2.0,5.0,30.0,53.0,31.0,154.0,391,473,139.0
5829,2016-10-03 10:40:00,2.0,5.0,31.0,53.0,32.0,154.0,392,484,134.0
5831,2016-10-03 10:50:00,2.0,5.0,31.0,51.0,32.0,158.0,390,498,134.0
5833,2016-10-03 11:00:00,2.0,5.0,32.0,51.0,33.0,167.0,402,511,133.0
...,...,...,...,...,...,...,...,...,...,...
168432,2025-01-05 06:50:00,75.0,51.0,296.0,571.0,878.0,757.0,9927,1101,5815.0
168433,2025-01-05 07:00:00,73.0,54.0,302.0,574.0,831.0,790.0,9985,1133,5618.0
168434,2025-01-05 07:10:00,71.0,56.0,314.0,574.0,782.0,822.0,10061,1179,5406.0
168435,2025-01-05 07:20:00,70.0,61.0,326.0,574.0,734.0,853.0,10152,1224,5259.0


In [11]:
df_combined_10min = df_combined_10min.drop_duplicates().reset_index(drop=True)
df_combined_10min = df_combined_10min.loc[df_combined_10min["Timestamp"] < pd.to_datetime("2024-06-12 16:50:00")]
df_combined_10min.to_csv("../data/old_data_10min_only.csv", index=False)

In [12]:
df_combined_10min

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2016-10-03 10:20:00,2.0,5.0,29.0,55.0,31.0,153.0,386,465,142.0
1,2016-10-03 10:30:00,2.0,5.0,30.0,53.0,31.0,154.0,391,473,139.0
2,2016-10-03 10:40:00,2.0,5.0,31.0,53.0,32.0,154.0,392,484,134.0
3,2016-10-03 10:50:00,2.0,5.0,31.0,51.0,32.0,158.0,390,498,134.0
4,2016-10-03 11:00:00,2.0,5.0,32.0,51.0,33.0,167.0,402,511,133.0
...,...,...,...,...,...,...,...,...,...,...
31830,2024-06-09 14:10:00,39.0,110.0,829.0,319.0,1061.0,1511.0,11142,7316,2713.0
31831,2024-06-09 14:20:00,42.0,110.0,823.0,307.0,1164.0,1529.0,11112,7385,2863.0
31832,2024-06-09 14:30:00,44.0,110.0,820.0,288.0,1199.0,1546.0,11040,7394,2978.0
31833,2024-06-09 14:40:00,48.0,110.0,820.0,276.0,1236.0,1546.0,10905,7434,3132.0


In [15]:
df_combined_10min_dedupe = df_combined_10min.copy()
df_combined_10min_dedupe["nan_count"] = df_combined_10min_dedupe.isnull().sum(axis=1)
df_combined_10min_dedupe = df_combined_10min_dedupe.sort_values(by=["Timestamp", "nan_count"]).drop_duplicates(subset="Timestamp", keep="first").reset_index(drop=True)
df_combined_10min_dedupe = df_combined_10min_dedupe.drop(columns=["nan_count"])
df_combined_10min_dedupe

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2016-10-03 10:20:00,2.0,5.0,29.0,55.0,31.0,153.0,386,465,142.0
1,2016-10-03 10:30:00,2.0,5.0,30.0,53.0,31.0,154.0,391,473,139.0
2,2016-10-03 10:40:00,2.0,5.0,31.0,53.0,32.0,154.0,392,484,134.0
3,2016-10-03 10:50:00,2.0,5.0,31.0,51.0,32.0,158.0,390,498,134.0
4,2016-10-03 11:00:00,2.0,5.0,32.0,51.0,33.0,167.0,402,511,133.0
...,...,...,...,...,...,...,...,...,...,...
31720,2024-06-09 14:10:00,39.0,110.0,829.0,319.0,1061.0,1511.0,11142,7316,2713.0
31721,2024-06-09 14:20:00,42.0,110.0,823.0,307.0,1164.0,1529.0,11112,7385,2863.0
31722,2024-06-09 14:30:00,44.0,110.0,820.0,288.0,1199.0,1546.0,11040,7394,2978.0
31723,2024-06-09 14:40:00,48.0,110.0,820.0,276.0,1236.0,1546.0,10905,7434,3132.0


In [10]:
bandwidths = pd.read_csv("../data/bandwidths.csv", parse_dates=["Timestamp"])
bandwidths

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2024-06-12 16:50:00,63.0,124.0,932.0,176.0,1152.0,1948.0,6941.0,7953.0,4087.0
1,2024-06-12 17:00:00,62.0,122.0,932.0,176.0,1152.0,1902.0,6444.0,7941.0,4087.0
2,2024-06-12 17:10:00,63.0,124.0,932.0,177.0,1162.0,1931.0,6444.0,7953.0,4130.0
3,2024-06-12 17:20:00,62.0,122.0,943.0,180.0,1175.0,1931.0,6025.0,8061.0,4206.0
4,2024-06-12 17:30:00,61.0,120.0,943.0,180.0,1175.0,1950.0,5663.0,8061.0,4219.0
...,...,...,...,...,...,...,...,...,...,...
29801,2025-01-05 15:40:00,52.0,132.0,900.0,192.0,1636.0,2222.0,8622.0,8655.0,2857.0
29802,2025-01-05 15:50:00,59.0,132.0,903.0,184.0,1689.0,2207.0,8218.0,8657.0,2988.0
29803,2025-01-05 16:00:00,63.0,133.0,903.0,175.0,1767.0,2207.0,7947.0,8739.0,3168.0
29804,2025-01-05 16:10:00,66.0,133.0,900.0,172.0,1778.0,2207.0,7655.0,8811.0,3342.0


In [16]:
combined = pd.concat([bandwidths, df_combined_10min_dedupe], axis=0, ignore_index=True).reset_index(drop=True)
combined = combined.set_index("Timestamp").resample("10min").asfreq().reset_index()
combined

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2016-10-03 10:20:00,2.0,5.0,29.0,55.0,31.0,153.0,386.0,465.0,142.0
1,2016-10-03 10:30:00,2.0,5.0,30.0,53.0,31.0,154.0,391.0,473.0,139.0
2,2016-10-03 10:40:00,2.0,5.0,31.0,53.0,32.0,154.0,392.0,484.0,134.0
3,2016-10-03 10:50:00,2.0,5.0,31.0,51.0,32.0,158.0,390.0,498.0,134.0
4,2016-10-03 11:00:00,2.0,5.0,32.0,51.0,33.0,167.0,402.0,511.0,133.0
...,...,...,...,...,...,...,...,...,...,...
434336,2025-01-05 15:40:00,52.0,132.0,900.0,192.0,1636.0,2222.0,8622.0,8655.0,2857.0
434337,2025-01-05 15:50:00,59.0,132.0,903.0,184.0,1689.0,2207.0,8218.0,8657.0,2988.0
434338,2025-01-05 16:00:00,63.0,133.0,903.0,175.0,1767.0,2207.0,7947.0,8739.0,3168.0
434339,2025-01-05 16:10:00,66.0,133.0,900.0,172.0,1778.0,2207.0,7655.0,8811.0,3342.0


In [17]:
regions_all = [
    "Central America",
    "Africa",
    "Middle East",
    "Oceania",
    "South America",
    "Russia",
    "Asia",
    "Europe",
    "North America",
]
combined[regions_all] = combined[regions_all].astype("Int64")

In [18]:
combined

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2016-10-03 10:20:00,2,5,29,55,31,153,386,465,142
1,2016-10-03 10:30:00,2,5,30,53,31,154,391,473,139
2,2016-10-03 10:40:00,2,5,31,53,32,154,392,484,134
3,2016-10-03 10:50:00,2,5,31,51,32,158,390,498,134
4,2016-10-03 11:00:00,2,5,32,51,33,167,402,511,133
...,...,...,...,...,...,...,...,...,...,...
434336,2025-01-05 15:40:00,52,132,900,192,1636,2222,8622,8655,2857
434337,2025-01-05 15:50:00,59,132,903,184,1689,2207,8218,8657,2988
434338,2025-01-05 16:00:00,63,133,903,175,1767,2207,7947,8739,3168
434339,2025-01-05 16:10:00,66,133,900,172,1778,2207,7655,8811,3342


In [19]:
combined.to_csv("../data/bandwidths.csv", index=False)